# Replication of results from Anderson et al. 2016

## Initializing

Notes :
- sampling frequency
- initial parameters (grid search, iterative)


In [1]:
import numpy as np
import scipy.io
import math
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import pyhsmm_mvpa as hsmm

import os

%load_ext autoreload
%autoreload 2


# Initial fit

Reading the data

In [2]:
mat = scipy.io.loadmat('matlab/newbumps/starter.mat')

data = np.array(mat['normedscore10'])
starts = np.array(mat['x'][:,0]) -1#correcting to be 0 indexed
ends = np.array(mat['y'][:,0])-1#correcting to be 0 indexed
subjects = np.array(mat['subjects'])-1 #correcting to be 0 indexed

testing single fit

In [4]:
init = hsmm.hsmm(data, starts, ends)

In [44]:
initial_estimates = init.fit_single(1)

Estimating parameters


/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:144: UserWarning: Using default parameters value for magnitudes and gamma
  warnings.warn('Using default parameters value for magnitudes and gamma')


Parameters estimated


In [45]:
initial_estimates

<xarray.Dataset>
Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                  trial: 18271)
Dimensions without coordinates: stage, params, component, bump, samples, trial
Data variables:
    likelihoods  float64 -8.769e+04
    parameters   (stage, params) float64 2.0 33.46 2.0 17.67
    magnitudes   (component, bump) float64 -1.219 0.4162 ... 0.1688 -0.05549
    eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [46]:
init.fit_single(1,parameters=initial_estimates.parameters.values, magnitudes=initial_estimates.magnitudes.values)

Estimating parameters
Parameters estimated


<xarray.Dataset>
Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                  trial: 18271)
Dimensions without coordinates: stage, params, component, bump, samples, trial
Data variables:
    likelihoods  float64 -8.769e+04
    parameters   (stage, params) float64 2.0 33.46 2.0 17.67
    magnitudes   (component, bump) float64 -1.219 0.4162 ... 0.1688 -0.05549
    eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

## Iterative fit

In [8]:
import time

start = time.time()
iterative_estimates = init.fit_iterative(2)
print(iterative_estimates)
end = time.time()
print(end - start)

/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:153: UserWarning: Using default parameters value for magnitudes and gamma
  


<xarray.Dataset>
Dimensions:     (n_bumps: 2, stage: 3, params: 2, component: 10, bump: 2)
Dimensions without coordinates: n_bumps, stage, params, component, bump
Data variables:
    likelihood  (n_bumps) float64 -8.766e+04 -6.729e+04
    parameters  (n_bumps, stage, params) float64 2.0 32.9 2.0 ... 2.0 16.92
    magnitudes  (n_bumps, component, bump) float64 -1.236 nan ... -0.07719
152.8647484779358


Parallel iterative fit

In [10]:
import multiprocessing 

cpus = multiprocessing.cpu_count()
start = time.time()
with multiprocessing.Pool(processes=cpus) as pool:
    results = pool.map(init.fit_single, np.arange(1,3))
end = time.time()
print(end - start)

Estimating parameters for 1 bumps model
Estimating parameters for 2 bumps model
Parameters estimated for 1 bumps model
Parameters estimated for 2 bumps model
123.11889219284058


In [12]:
start = time.time()
with multiprocessing.Pool(processes=cpus) as pool:
    results = pool.map(init.fit_single, np.arange(1,8))# 8 bump mode fail?
end = time.time()
print(end - start)

Estimating parameters for 1 bumps model
Estimating parameters for 2 bumps model
Estimating parameters for 3 bumps model
Estimating parameters for 4 bumps model
Estimating parameters for 5 bumps model
Estimating parameters for 6 bumps model
Estimating parameters for 7 bumps model
Parameters estimated for 1 bumps model
Parameters estimated for 3 bumps model
Parameters estimated for 2 bumps model
Parameters estimated for 4 bumps model
Parameters estimated for 5 bumps model
Parameters estimated for 6 bumps model
Parameters estimated for 7 bumps model
851.3596642017365


In [13]:
start = time.time()
iterative_estimates = init.fit_iterative(7)
print(iterative_estimates)
end = time.time()
print(end - start)

<xarray.Dataset>
Dimensions:     (n_bumps: 7, stage: 8, params: 2, component: 10, bump: 7)
Dimensions without coordinates: n_bumps, stage, params, component, bump
Data variables:
    likelihood  (n_bumps) float64 -8.766e+04 -6.729e+04 ... -4.315e+04
    parameters  (n_bumps, stage, params) float64 2.0 32.9 2.0 ... 2.0 2.906
    magnitudes  (n_bumps, component, bump) float64 -1.236 nan ... -0.1522
1547.0548796653748


Parallel iterative fit

In [71]:
results

[<xarray.Dataset>
 Dimensions:      (stage: 2, params: 2, component: 10, bump: 1, samples: 300,
                   trial: 18271)
 Dimensions without coordinates: stage, params, component, bump, samples, trial
 Data variables:
     likelihoods  float64 -8.766e+04
     parameters   (stage, params) float64 2.0 32.9 2.0 18.22
     magnitudes   (component, bump) float64 -1.236 0.3476 ... 0.134 -0.05735
     eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0,
 <xarray.Dataset>
 Dimensions:      (stage: 3, params: 2, component: 10, bump: 2, samples: 300,
                   trial: 18271)
 Dimensions without coordinates: stage, params, component, bump, samples, trial
 Data variables:
     likelihoods  float64 -6.729e+04
     parameters   (stage, params) float64 2.0 10.33 2.0 21.63 2.0 16.92
     magnitudes   (component, bump) float64 0.8706 -1.085 ... 0.04723 -0.07719
     eventprobs   (samples, trial, bump) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0]

In [55]:
iterative_estimates = init.fit_iterative(2)

/home/gweindel/owncloud/projects/RUGUU/pyhsmm-mvpa/pyhsmm_mvpa.py:150: UserWarning: Using default parameters value for magnitudes and gamma
  magnitudes = np.zeros((self.n_dims,n_bumps))


In [64]:
iterative_estimates = init.fit_iterative(8,parameters=iterative_estimates.parameters.values, magnitudes=iterative_estimates.magnitudes.values)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [63]:
iterative_estimates

<xarray.Dataset>
Dimensions:     (n_bumps: 2, stage: 3, params: 2, component: 10, bump: 2)
Dimensions without coordinates: n_bumps, stage, params, component, bump
Data variables:
    likelihood  (n_bumps) float64 -8.766e+04 -6.729e+04
    parameters  (n_bumps, stage, params) float64 2.0 32.9 2.0 ... 2.0 16.92
    magnitudes  (n_bumps, component, bump) float64 -1.236 nan ... -0.07719